### import 및 기초작업
- 기본은 직접입력, (최초 or 신고당시)이자채권 또는 특수한 경우 입금순서적용(발견하는대로 리스트에 추가)
- 신복 입금원금은 전산에서 원금-비용 순으로 차감
- 신복 이자, 기타채무는 전산에서 미수-연체 순으로 차감

In [1]:
import os, re
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
from datetime import datetime, timedelta
import warnings
warnings.simplefilter("ignore")
pd.options.display.float_format = '{:,.0f}'.format

def get_previous_weekday_yymmdd(date):
    """날짜를 받아 이전 마지막 평일 날짜 yymmdd return"""
    # 전일
    date = date - timedelta(days=1)
    # 전일부터 하루씩 차감하면서 평일이면 return
    while date.weekday() > 4:  # 월요일은 0이고, 금요일은 4
        date -= timedelta(days=1)
    return date.strftime('%y%m%d')

def 파일명(폴더:str, 키워드:str, 부정키워드="") :
    file_list = os.listdir(폴더)
    if 부정키워드 == "" : 
        fn = [file for file in file_list if re.search(키워드, file)]
    else : 
        fn = [file for file in file_list if (re.search(키워드, file)!=None) & (re.search(부정키워드,file)==None)]
    if len(fn) == 1 :
        return join(폴더,fn[0])
    else :
        print("조건을 만족하는 파일이 둘 이상입니다.")
        
def 입금항목현재항목(입금순서:str) :
    """입금순서를 받아 입금항목명리스트와 현재항목명리스트 (2차원)를 반환"""
    입금순서 = re.sub('\d',"",입금순서)
    입금순서 = re.sub('법비용', "입금법비용", 입금순서)
    입금순서 = re.sub('미수', "입금미수이자", 입금순서)
    입금순서 = re.sub('연체', "입금연체이자", 입금순서)
    입금순서 = re.sub('원금', "입금원금", 입금순서)
    입금항목들 = 입금순서.split(' ')
    현재항목들 = [re.sub('입금', '현재', item) for item in 입금항목들]
    return 입금항목들, 현재항목들
        
        
credit_deposit_dtype = {"No.":str, "채무자키" : str, "계좌키":str, "담당자키":str, "타입금키":str, "납입회차":str, "계좌번호":str}
account_dtype = {'채무자키':str, '계좌키':str, '계좌번호':str}

In [9]:
# 어차피 계좌정보 읽으려면 입금창에서 키매칭 작업 해야함.
# 입금창에서 최초원금, 현재금액들 불어와 준다면 더욱 더 키매칭 작업을 하는게 효율적
# 입금경로1 : 신용회복변제금상환 파일 양식에 금액 조정 해주고, 입금순서 컬럼추가 > 신용회복 입금
# 입금경로2 : 신용회복 to 항목별입금 > 유틸리티 항목별입금
# 중복건 입금 순서는 No.도 아니고 회차도 아니고 행순서대로 먼저 적용됨. 따라서 회차별로 정렬하면 됨

################################
company = "솔림" # 솔림 or 대성
workday = "" # 특별히 원하는 날짜 없으면 빈문자열로 남겨둘것
입금순서기본값 = "1법비용 2미수 3연체 4원금"
################################

# 입금일(최근 평일 or 선택일)
if workday == "" : 
    # 오늘 이전 중 가장 가까운 평일 날짜 가져오기
    workday = get_previous_weekday_yymmdd(datetime.today())
    
year = "20"+workday[:2] # 2024

# 경로 및 파일명 : 키매칭 파일, 계좌 
wd = join(r"D:\3.자산\신용회복\신용회복 to 일반입금", company, year)
fn_credit_deposit = workday+"_일괄입금구분_"+company+".xlsx"
fn_account = workday+"_계좌_"+company+".xlsx"
print(workday)

240528


### 재작업 시작점

##### 1.파일읽기

In [13]:
# 오류 아니라서 직접입력 아니라 입금순서 적용할 필요가 있는 건 : 최초금액은 있으나, 채권신고당시 현재원금이 0원 / 허위신고의심but최초금액수정안하기로한것
temp = pd.read_excel(r'D:\3.자산\신용회복\신용회복 to 일반입금\입금순서적용할건.xlsx',sheet_name=company, dtype={"채무자키":str, "계좌키":str})
입금순서적용계좌키 = temp.계좌키.to_list()

print("입금순서 적용할 계좌키 수 : ", len(입금순서적용계좌키))

cre_dep_ori = pd.read_excel(join(wd, fn_credit_deposit), dtype = credit_deposit_dtype).fillna("")
account_ori = pd.read_excel(join(wd, fn_account), dtype = account_dtype).fillna("")

# 타입금키에 중복있는지 체크
print("타입금키 중복체크 : False만 있어야")
print(cre_dep_ori.타입금키.duplicated().value_counts())

# 입금법비용 칼럼 만들기, 입금비고에 회차적기
cre_dep_ori["입금법비용"] = 0
cre_dep_ori["입금비고"] = cre_dep_ori["납입회차"]

입금순서 적용할 계좌키 수 :  57
타입금키 중복체크 : False만 있어야
타입금키
False    1533
Name: count, dtype: int64


##### 2.재분배 및 오류체크

In [14]:
cre_dep = cre_dep_ori.copy()
account = account_ori.copy()[["계좌키", "최초원금", "현재원금", "현재법비용", "현재미수이자", "현재연체이자", "현재합계"]]
# 입금파일 계좌키,납입회차 순으로 정렬
cre_dep.sort_values(["계좌키", "납입회차"], inplace=True)

# '계좌키' 열에서 각 값의 누적 빈도를 계산하여 새로운 열 '계좌키_누적빈도'에 추가
cre_dep['계좌키빈도'] = cre_dep.groupby('계좌키').cumcount() + 1

# 최초금액 및 현재금액 병합 - 인덱스 새로 부여됨
cre_dep = cre_dep.merge(account, how='left')
# 변수에 담으면 인덱스가 재설정되긴 하던데, 혹시 모르니 확실하게. 아래 반복문에서 i-1을 하려면 필수!
cre_dep.reset_index(drop=True, inplace=True)

# 직접입력 대신 입금순서 작성하기 - 이자채권류 : 최초원금 10원미만, 신고당시 이자채권--------------------
# 그러나 결국은 분배를 내가 해줘야 항목별입금을 사용할 수 있다.
입금항목들, 현재항목들 = 입금항목현재항목(입금순서기본값)
입금순서cond = (cre_dep.계좌키.isin(입금순서적용계좌키) | (cre_dep.최초원금<10) )
cre_dep["입금순서"] = np.where(입금순서cond, 입금순서기본값, cre_dep.입금순서)
#--------------------------------------------------------------------------------------------------

# 오류열 만들고 이미 오류난 거 체크
cond = (cre_dep.현재원금<0) | (cre_dep.현재법비용<0) | (cre_dep.현재미수이자<0) | (cre_dep.현재연체이자<0)
cre_dep["오류"] = np.where(cond, "오류", "")
for i, v in tqdm(cre_dep.iterrows(), total = len(cre_dep)) : 
    # 단건 또는 다건 중 가장 앞선 회차
    if v.계좌키빈도 == 1 :
        # 입금순서 : 분배는 나우리로 하므로 합계만 따지면 됨
        if v.입금순서 != "직접입력" :
            # 오류 체크
            if v.입금합계 > v.현재합계 : cre_dep.loc[i, "오류"] = "오류"
            # 정상건 : 입금분배 - 잔여합계가 0이 될때까지
            else :
                # 나우리 분배건 초기화
                cre_dep.loc[i,["입금원금", "입금법비용", "입금미수이자", "입금연체이자"]] = 0
                
                # 분배 재작업
                잔여합계 = v.입금합계
                for 입금항목, 현재항목 in zip(입금항목들, 현재항목들) : 
                    항목입금액 = min(cre_dep.loc[i,현재항목], 잔여합계)
                    # 항목별 입금액 배정
                    cre_dep.loc[i, 입금항목] = 항목입금액
                    # 잔여합계금 재계산
                    잔여합계 -= 항목입금액
                    # 잔여금이 0이면 종료
                    if 잔여합계 == 0 : break
                
        # 직접입력 : 입금미수이자와 연체이자는 나우리가 자동으로 나눈다. 혹시 -값이 있는지만 확인
        # 입금원금을 원금-비용 순으로 바꿔준다.(순서는 상관없다. 실효되면 원상회복 할거니까. 다만 실효전 부채증명서 발급시 비용은 감면이 안 돼서 오히려 유리)
        else : 
            # 원금 오류 체크 - 이자와 별도로 해야함
            if (v.입금원금 > v.현재원금) :
                입금법비용 = v.입금원금 - v.현재원금
                if 입금법비용 > v.현재법비용 : 
                    cre_dep.loc[i, "오류"] = "오류"
                else : 
                    cre_dep.loc[i, "입금원금"] = cre_dep.loc[i, "현재원금"]
                    cre_dep.loc[i, "입금법비용"] = 입금법비용

            # 이자 오류 체크 
            if (v.입금미수이자 > v.현재미수이자) or (v.입금연체이자 > v.현재연체이자) : 
                cre_dep.loc[i, "오류"] = "오류"
            
            
            
    # 다건 중 나중 회차 
    # 현재금액 = 전회차현재금액 - 전회차 입금액으로 수정필요, 정렬 했으므로 이전행의 값을 불러오면 됨
    else : 
        
        # 현재금액 수정------------------------------------------------------
        cre_dep.loc[i, "현재합계"] = cre_dep.loc[i-1, "현재합계"] - cre_dep.loc[i-1, "입금합계"]
            
        for 입금항목, 현재항목 in zip(입금항목들, 현재항목들) : 
            cre_dep.loc[i, 현재항목] = cre_dep.loc[i-1, 현재항목] - cre_dep.loc[i-1, 입금항목]
        # ------------------------------------------------------------------
            
        # 입금순서 : 분배는 나우리로 하므로 합계만 따지면 됨
        if v.입금순서 != "직접입력" : 
            # 오류 체크
            if v.입금합계 > cre_dep.loc[i, "현재합계"] : 
                cre_dep.loc[i, "오류"] = "오류"
            # 정상건 입금분배
            else :
                # 나우리 분배건 초기화
                cre_dep.loc[i,["입금원금", "입금법비용", "입금미수이자", "입금연체이자"]] = 0
                
                # 분배 재작업
                잔여합계 = v.입금합계
                for 입금항목, 현재항목 in zip(입금항목들, 현재항목들) : 
                    항목입금액 = min(cre_dep.loc[i,현재항목], 잔여합계)
                    # 항목별 입금액 배정
                    cre_dep.loc[i, 입금항목] = 항목입금액
                    # 잔여합계금 재계산
                    잔여합계 -= 항목입금액
                    # 잔여금이 0이면 종료
                    if 잔여합계 == 0 : break
        
        # 직접입력의 경우(나우리분배) : 나우리분배는 미수이자 연체이자로만 분배되므로 이것들만 체크 및 재분배하면 됨
        else :
            
            # 미수와 연체 재분배(체크할 오류x)
            실현재미수이자 = cre_dep.loc[i,"현재미수이자"]
            if 실현재미수이자 - v.입금미수이자 < 0 : 
                cre_dep.loc[i, "입금미수이자"] = 실현재미수이자 # 실제 남은 현재미수이자만큼만 입금, 나머지는 연체이자로
                cre_dep.loc[i, "입금연체이자"] = v.입금연체이자 - (실현재미수이자 - v.입금미수이자) # 차액 추가 -(-) = +
            
            
            # 원금 비용 재분배 및 오류체크 - 이자와 별도로 해야함
            if (v.입금원금 > v.현재원금) :
                입금법비용 = v.입금원금 - v.현재원금
                if 입금법비용 > v.현재법비용 : 
                    cre_dep.loc[i, "오류"] = "오류"
                else : 
                    cre_dep.loc[i, "입금원금"] = cre_dep.loc[i, "현재원금"]
                    cre_dep.loc[i, "입금법비용"] = 입금법비용

            # 이자 오류 체크 
            if (cre_dep.loc[i, "입금연체이자"] > cre_dep.loc[i, "현재연체이자"]) : 
                cre_dep.loc[i, "오류"] = "오류"
            
                     

# 원본과 차이점 기록 및 재배정 오류 확인
cre_dep = cre_dep.merge(cre_dep_ori[["타입금키", "입금원금","입금법비용", "입금미수이자", "입금연체이자", "입금합계"]], on="타입금키", suffixes=["", "_원본"])
cre_dep["원금비교"] = cre_dep.입금원금 - cre_dep.입금원금_원본
cre_dep["법비용비교"] = cre_dep.입금법비용 - cre_dep.입금법비용_원본
cre_dep["미수비교"] = cre_dep.입금미수이자 - cre_dep.입금미수이자_원본
cre_dep["연체비교"] = cre_dep.입금연체이자 - cre_dep.입금연체이자_원본
cre_dep["합계비교"] = cre_dep.입금합계 - cre_dep.입금합계_원본

# 수정항목있음
수정여부_cond = (cre_dep.원금비교 != 0) |(cre_dep.법비용비교 != 0)|(cre_dep.미수비교 != 0)|(cre_dep.연체비교 != 0)|(cre_dep.합계비교 != 0)
cre_dep["수정여부"] = np.where(수정여부_cond, "수정", "미수정")

# 합계 다른 경우 : 무언가 잘못됐다.
print("오류!!! 합계비교 0이 아닌 건")
display(cre_dep.query('합계비교 != 0'))

# # 원본열 삭제 - 비교하러 보는 거니 삭제하지 말자
# cre_dep.drop(columns=["입금원금_원본", "입금비용_원본", "입금미수_원본", "입금연체_원본","입금합계_원본"])

100%|██████████| 1533/1533 [00:00<00:00, 20238.68it/s]

오류!!! 합계비교 0이 아닌 건


,No.,채무자키,계좌키,담당자키,입금구분,입금순서,입금기타,타입금키,입금메모,리파이낸싱,...,입금법비용_원본,입금미수이자_원본,입금연체이자_원본,입금합계_원본,원금비교,법비용비교,미수비교,연체비교,합계비교,수정여부


##### 3. 파일저장

In [15]:
#-----------------------------------------------------------
항목별입금 = True # True로 할 것!!!! 
# False : 신복양식, True : 일반입금(항목별입금) 양식
# 신용회복은 중복건 미수,연체이자 분배 재작업한 것인 무용지물이 되어버리니 쓸수없다. 전산 입금파일 경로도 바뀌어서 귀찮을테고..
#-----------------------------------------------------------

# 파일 저장 후 오류건 전산 수정 > 계좌 다시 다운
오류건수 = len(cre_dep.query('오류 != "" '))
if 오류건수 > 0 : 
    print('오류건 전산 수정 후 계좌 데이터를 다시 다운로드 하고 처음부터 재작업하세요')
    cre_dep.to_excel(join(wd, workday + "_오류체크_"+company+".xlsx"), index=False)
else : 
    print('오류건이 없습니다.')
    # 차이점 나중에 확인하기 편하게 최종본(오류없는건) 출력
    cre_dep.to_excel(join(wd, workday + "_일괄입금구분_최종본_"+company+".xlsx"), index=False)
    
    # 입금순서 모두 직접입력으로 바꾸고 출력
    cre_dep.입금순서 = "직접입력"
    if 항목별입금 : 
        print('항목별입금양식을 출력합니다.')
        # 일반입금파일 작성
        cols = ["채무자키", "계좌키", "입금일", "입금합계",'입금원금','입금법비용','입금미수이자', '입금연체이자',
                "입금구분", "입금자", "입금자구분", "입금은행", "입금계좌번호", "입금순서", "입금메모", "입금비고"]
        항목별입금 = cre_dep[cols]
        항목별입금.to_excel(join(wd, workday + "__항목별입금_"+company+".xlsx"), index=False)
        ##### 일반입금은 타입금키 있으나 항목별은 없음. 일반입금은 입금합계가 아니라 입금액임
    else : 
        print('신용회복입금양식을 출력합니다.')
        # 신용회복양식으로 작성
        cols = ['No.', '납입회차', '일시납감면율', '입금일', '지점명', '입금자',
        '주민번호', '지원구분', '입금자구분', '주채무자주민번호', '주채무자', '계좌번호', 
        '입금원금', '입금미수이자', '입금연체이자', '입금합계', '확정채권금액',  '입금계좌번호', '지로코드', '입금순서']
        신용회복입금 = cre_dep[cols]
        # 컬럼명은 안 바꿔도 된다. 대신 순서가 바뀌는 경우 맞춰줘야. 신용회복입금.rename(columns={'납입회차':'회차', '입금일':'처리일자'}, inplace=True)
        with pd.ExcelWriter(join(wd, workday + "__신용회복입금_"+company+".xlsx")) as writer:
            신용회복입금.to_excel(writer, index=False, startrow=2)
         
# 대성 : 항목별입금으로 작업해보자. 입금은 잘 됨. 신복 스케쥴에도 잘 반영되는지 확인필요 
# 솔림 : 신복수정하는 걸로 작업해보자.

오류건이 없습니다.
항목별입금양식을 출력합니다.


###### 오류 무시 OR 오류 간단 수정완료

In [ ]:
# 오류건 정정 후 굳이 계좌 정보 다시 받을 필요 없을때
cols = ["채무자키", "계좌키", "입금일", "입금합계",'입금원금','입금법비용','입금미수이자', '입금연체이자',
                "입금구분", "입금자", "입금자구분", "입금은행", "입금계좌번호", "입금순서", "입금메모", "입금비고"]
항목별입금 = cre_dep[cols]
항목별입금.to_excel(join(wd, workday + "__항목별입금_"+company+".xlsx"), index=False)